# Dask Array

Материалы:
* Макрушин С.В. Лекция 11: Dask
* https://docs.dask.org/en/latest/array.html
* JESSE C. DANIEL. Data Science with Python and Dask. 

## Задачи для совместного разбора

In [1]:
import dask
import dask.array as da
import numpy as np

1. Создайте массив размерностью 1000 на 300000, заполненный числами из стандартного нормального распределения. Исследуйте основные характеристики полученного массива.

In [13]:
%%time
arr_np = np.random.normal(0, 1, size=(1000,300_000))

ValueError: array is too big; `arr.size * arr.dtype.itemsize` is larger than the maximum possible size.

In [ ]:
%%time
arr_da = da.random.normal(0, 1, size=(1000,300_000), chunks=(1000,30_000))
arr_da

Wall time: 2.02 ms


dask.array<normal, shape=(1000, 300000), dtype=float64, chunksize=(1000, 30000), chunktype=numpy.ndarray>

2. Посчитайте сумму квадратов элементов массива, созданного в задаче 1. Создайте массив `np.array` такого же размера и сравните скорость решения задачи с использование `da.array` и `np.array`

In [ ]:
%%time
np.power(arr_np, 2).sum()

Wall time: 10 s


300024213.36711586

In [ ]:
%%time
da.power(arr_da, 2).sum().compute()

Wall time: 4.49 s


300002051.5237932

In [ ]:
# А ТАК НЕ НАДО ДЕЛАТЬ
# %%time
# np.power(arr_da.compute(), 2).sum()

3. Визуализируйте граф вычислений для задачи 2.

In [ ]:
da.power(arr_da, 2).sum().vizualize()

## Лабораторная работа 11

In [11]:
import dask.array as da
import h5py
import numpy as np
import pandas

1. Считайте датасет `recipe` из файла `minutes_n_ingredients_full.hdf5` в виде `dask.array`. Укажите аргумент `chunks=(100_000, 3)` при создании массива. Выведите на экран основную информацию о массиве.

In [2]:
f = h5py.File('../data/minutes_n_ingredients_full.hdf5')
da_recipe = da.from_array(f['recipe'], chunks=(100_000, 3))
da_recipe

dask.array<array, shape=(2231637, 3), dtype=int64, chunksize=(100000, 3), chunktype=numpy.ndarray>

2. Вычислите среднее значение по каждому столбцу, кроме первого. 

In [3]:
%%time
da_recipe[:,1].mean().compute()

Wall time: 192 ms


1004.2080517575215

In [4]:
%%time
da_recipe[:,2].mean().compute()

Wall time: 61.9 ms


5.419800800936711

3. Исследуйте, как влияет значение аргумента `chunks` при создании `dask.array` на скорость выполнения операции поиска среднего. 

* *chunks=(100_000, 3)* = 67.8 ms & 69.8 ms
* *chunks=(1_000_000, 3)* = 62.7 ms & 59.3 ms
* *chunks=(10_000, 3)* = 156 ms & 148 ms

4. Выберите рецепты, время выполнения которых меньше медианного значения

In [5]:
mediana = da.median(da_recipe[:, 1], axis=0).compute()
da_recipe[da_recipe[:, 1]<mediana].compute()

array([[1089012,      23,       5],
       [1428572,       0,       5],
       [1400250,      24,       1],
       ...,
       [1029131,      19,       4],
       [1700703,       1,       1],
       [ 713836,       0,       9]], dtype=int64)

5. Посчитайте количество каждого из возможных значений кол-ва ингредиентов

In [ ]:
value, count = da.unique(da_recipe, return_counts=True)
for v, c in zip(value.compute(), count.compute()):
    print(f'{v} -> {c}')

6. Найдите максимальную продолжительность рецепта. Ограничьте максимальную продолжительность рецептов сверху значением, равному 75% квантилю.

In [6]:
da_recipe[:,1].max().compute()

2147483647

In [24]:
quantile = da_recipe[:,1].to_dask_dataframe().quantile(q=0.75, method='default').compute()
quantile

49.0

In [25]:
da_recipe_over = da_recipe[da_recipe[:,1]>quantile].compute()
da_recipe[da_recipe[:,1]>quantile, 1] = quantile
da_recipe[:,1].max().compute()

49

7. Создайте массив `dask.array` из 2 чисел, содержащих ваши предпочтения относительно времени выполнения рецепта и кол-ва ингредиентов. Найдите наиболее похожий (в смысле $L_1$) рецепт из имеющихся в датасете.

In [30]:
da_my = da.from_array([30, 4])
da_recipe[da.fabs(da_my - da_recipe[:, 1:]).sum(axis=1).argmin()].compute()

array([847357,     30,      4], dtype=int64)

8. Работая с исходным файлом в формате `hdf5`, реализуйте алгоритм подсчета среднего значения в блочной форме и вычислите с его помощью среднее значение второго столбца в массиве.

Блочный алгоритм вычислений состоит из двух частей:
1. загрузка фрагмента за фрагментом данных по `blocksize` элементов и проведение вычислений на этим фрагментом;
2. агрегация результатов вычислений на различных фрагментах для получения результата на уровне всего набора данных.

Важно: при работе с `h5py` в память загружаются не все элементы, а только те, которые запрашиваются в данный момент

In [58]:
f = h5py.File('../data/minutes_n_ingredients_full.hdf5')
recipe = f['recipe']
blocksize = 500_000
parts = np.array([])

for i in range(0, recipe.shape[0] + 1, blocksize):
    parts = np.append(parts, np.sum(recipe[i:i+blocksize, 1]))

np.sum(parts)/recipe.shape[0]

1004.2080517575215